# OpenAI proof-of-concept with labels merged

Demonstrating the use of the OpenAI API to analyse text toxicity and cyberbullying.

https://openai.com/api/

We are merging some labels to increase the performance.

Labels we have currently:
```
1. Racisme
2. Insulte (et Discrimination)
3. Obscénité
4. Sexualité explicite
5. Menace
6. Toxicité
7. Sexisme
```
**Suggestion:** We may merge some of them and group into 5:
```
1. Insult or Discrimination (Racisme + Insulte (et Discrimination))
2. Obscene or Sexual (Obscénité + Sexualité explicite)
3. Threats of Violence (Menace)
4. Cyberbullying or Toxic (Toxicité)
5. Insult or Discrimination (Sexisme)
```

In [1]:
import os
import time
import openai
import pandas as pd
from tqdm import tqdm

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def classify_text(input_text, retry=0):
    prompt = f"""The following is a list of text categories in French:

    1. Insult or Discrimination
    2. Obscene or Sexual
    3. Threats of Violence
    4. Cyberbullying or Toxic

    Please classify the following text as one of the above categories:
    "{input_text}"
    """
    
    # Get Prediction from OpenAI
    try:
        response = openai.Completion.create(
          model="text-davinci-003",
          prompt=prompt,
          temperature=0,
          max_tokens=20,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
    except Exception as e:
        if retry < 3:
            print("Error on classification. Retrying in 5 sec ",e)
            time.sleep(5)
            return classify_text(input_text, retry= retry+1)
        else:
            print("Error on classification. Reached retry limit ",e)
            return None
    
    # Parse Response
    label = response["choices"][0].text.strip()
    if "." in label:
        # Sometimes openai returns response as 1.Racisme
        label = label.split(".")[1].strip()
    
    if not label:
        label = None
    return label

## Prepare Data

In [4]:
# Read Data
df = pd.read_csv("safebear_data.csv")
df.label = df.label.replace({
    "Racisme":"Insult or Discrimination",
    "Insulte (et Discrimination)":"Insult or Discrimination",
    "Obscénité":"Obscene or Sexual",
    "Sexualité explicite":"Obscene or Sexual",
    "Menace":"Threats of Violence",
    "Toxicité":"Cyberbullying or Toxic",
    "Sexisme":"Insult or Discrimination"
})
print("Classes \n",df.label.unique())

Classes 
 ['Insult or Discrimination' 'Obscene or Sexual' 'Threats of Violence'
 'Cyberbullying or Toxic']


In [5]:
# Generate Predictions
predictions = []
for i,row in df.iterrows():
    predictions.append(classify_text(row.text))
    # to avoid ratelimiterror
    time.sleep(2)
        
df['prediction'] = predictions
df['correct'] = (df.label == df.prediction)

## Accuracy

In [6]:
accuracy = df['correct'].sum() / len(df) * 100
print(f"Correct Prediction / Total Item: {df['correct'].sum()} / {len(df)}")
print(f"Accuracy: {accuracy:.2f}%")

Correct Prediction / Total Item: 28 / 37
Accuracy: 75.68%


## Display Predictions

In [7]:
for i,row in df.iterrows():
    sign = "✅" if row.correct else "❌"
    print(f"Text: {row.text}")
    print(f"Prediction: {row.prediction}")    
    print(f"True Label: {row.label} {sign}\n")

Text: De toute façon t’es qu’un sale juif t’aurais du mourir en 45
Prediction: Insult or Discrimination
True Label: Insult or Discrimination ✅

Text: Au pire retourne à la mosquée sale arabe
Prediction: Insult or Discrimination
True Label: Insult or Discrimination ✅

Text: C’est pas l’heure d’aller bouffer des chenilles avec tes 29 femmes le négro ?
Prediction: Insult or Discrimination
True Label: Insult or Discrimination ✅

Text: Oh mon Dieu, je n’avais pas vu un Juif courir comme ça depuis les soldes du salon du cuir
Prediction: Insult or Discrimination
True Label: Insult or Discrimination ✅

Text: Mais nan j’irai pas en enfer j’suis pas black
Prediction: Insult or Discrimination
True Label: Insult or Discrimination ✅

Text: Pour vous les juifs le paradis c’est de trouver un cheveu dans votre soupe au resto pour vous faire rembourser
Prediction: Insult or Discrimination
True Label: Insult or Discrimination ✅

Text: Est-ce qu'on peut se débarrasser de tous les Mexicains ?
Prediction: 

## Custom Sentences: Neutral Performance

In [8]:
sentences = [
    "J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.",
    "La vidéo était mauvaise. Je ne l'aime pas.",
    "Les immigrés vivent avec nous dans notre pays.",
    "Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée."
]
for sentence in sentences:
    label = classify_text(sentence)
    print(f"Text: {sentence}")
    print(f"Prediction: {label}\n")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: None of the Above

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: None of the Above

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: None of the Above

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: None of the Above



## Custom Sentences: Difficult Examples

In [9]:
sentences = [
    "Je pense que les immigrés ne devraient pas exister dans notre pays.",    
    "Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.",
    "Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.",
    "Les hommes peuvent faire des travaux plus difficiles que les femmes.",
    "La présence des femmes dans le monde du travail est importante.",
]
for sentence in sentences:
    label = classify_text(sentence)
    print(f"Text: {sentence}")
    print(f"Prediction: {label}\n")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: Insult or Discrimination

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: None of the Above

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: None of the Above

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: None of the Above

Text: La présence des femmes dans le monde du travail est importante.
Prediction: None of the Above

